# Trabajo Final

###  CLASIFICACION CON RANDOM FOREST

### Universidad de Antioquia

### Facultad de Ingeniería

### Ingeniería de Sistemas

### Ude@ - 2018-I

#### Profesor: Antonio Tamayo Herrera

### Integrantes:

Nombre: David de Jesus Yepes Isaza

Cédula: 1020455542

Nombre: Rodrigo Antonio Porras Martinez

Cédula: 1039459720

### Clasificación en el dataset "Heart Disease Data Set" con RANDOM FOREST

La base de datos con las que se trabaja en el presente artículo fue extraída de  Cleveland Clinic Foundation y fue creada por V.A. Medical Center, Long Beach and Cleveland Clinic Foundation: Robert Detrano, M.D., Ph.D.

El set de datos de de Cleveland consta de 303 instancias, la variable de salida o etiqueta son 0 para la ausencia y los valores 1, 2, 3 y 4 para la presencia de enfermedades. 
 
El 54% de las instancias tienen ausencia de enfermedades y 46% tienen presencia de enfermedades del corazón. Para simplificar el problema se analiza como un problema biclase donde  “0” son personas sanas y “1” que corresponde a (1,2,3,4) son personas enfermas.

El objetivo es según las características de cada paciente identificar cuál de ellos presenta presencia de enfermedades y cuáles no.

La base de datos contiene 303 muestras con las 14 características siguientes:
1. N° 3 (edad)   
2. N° 4 (sexo) 
3. N° 9 (cp) tipo de dolor en el pecho
        Valor 1: angina típica
        Valor 2: angina atípica
        Valor 3: dolor no anginal
        Valor 4: asintomático  
4. N° 10 (trestbps) presión arterial en reposo
5. N° 12 (chol) colesterol sérico en mg / dl (miligramo por decilitro)
6. N° 16 (fbs) (azúcar en la sangre en ayunas > 120 mg / dl) (1 = verdadero, 0 = falso)
7. N° 19 (restecg)  resultados electrocardiográficos en reposo:
        Valor 0: normal
        Valor 1: tiene anormalidad de onda ST-T (inversiones de onda T y / o ST  elevación o depresión de > 0.05 mV)
        Valor 2: muestra hipertrofia ventricular izquierda probable o definitiva según los criterios de Estes  
8. N° 32 (thalach)  frecuencia cardíaca máxima lograda
9. N° 38 (exang) angina inducida por el ejercicio (1 = sí; 0 = no)
10. N° 40 (oldpeak) depresión del ST inducida por el ejercicio en relación con el reposo
11. N° 41 (pendiente) la pendiente del segmento ST de ejercicio máximo
        Valor 1: pendiente ascendente
        Valor 2: plano
        Valor 3: downsloping
12. N° 44 (ca) número de vasos principales (0-3) coloreados por fluoroscopia
13. N° 51 (thal) 3 = normal; 6 = defecto fijo; 7 = defecto reversible
14. N° 58 (num) (el atributo predicho) diagnóstico de enfermedad cardíaca (estado angiográfico de la enfermedad)
        - Valor 0: <50% de diámetro de estrechamiento
        - Valor 1:> 50% de diámetro de estrechamiento
    

###  Importacion de librerias y limpiado de datos

In [1]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize
from __future__ import division
from sklearn import tree
from sklearn.metrics import confusion_matrix
import time


def class_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):
        if y_e != y_r:
            err += 1

    return "accuracy: " + str(1-(err/np.size(y_est)))

def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):
        if y_e != y_r:
            err += 1
    return err/np.size(y_est)

clevelanddb = np.loadtxt('cleveland.txt',delimiter=',')  # Assuming comma-delimiter
print ("Dim de la base de datos original: " + str(np.shape(clevelanddb)))
DataBase = clevelanddb
#La base de datos cleveland.txt tiene algunos valores perdidos, estos se identifican con -9.0
#Los valores de salida mayores a 0 se considerarán 1 de esta forma la clasificacion será biclase 
j = 0
num_reset_values = 0

for i in range(0,np.size(clevelanddb,0)):
    if -9.0 == clevelanddb[i,13]:
        j+=1
        DataBase = np.delete(DataBase,i,0)
    elif clevelanddb[i,13] > 0:
        clevelanddb[i,13] = 1
        num_reset_values += 1

print ("\nHay " + str(j) + " valores perdidos en la variable de salida.")

print ("\nDim de la base de datos sin las muestras con variable de salida perdido " + str(np.shape(DataBase)))

print ("Se han reseteado a 1 "+str(num_reset_values)+" valores ya que eran mayores a 0")

#Ya hemos eliminado los registros con valor de la variable de salida perdido.

#Ahora vamos a imputar los valores perdidos en cada una de las características
print ("\nProcesando imputación de valores perdidos en las características . . .\n")

for k in range(0,np.size(DataBase,0)):
    for w in range(0,14):
        if -9.0 == DataBase[k,w]:
            DataBase[k,w] = round(np.mean(DataBase[:,w]))
        
print ("Imputación finalizada.\n")

hay_missed_values = False
for i in range(0,np.size(DataBase,0)):
    if -9.0 in DataBase[i,:]:
        hay_missed_values = True
if(hay_missed_values):
    print ("Hay valores perdidos")
else:
    print ("No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase")


Dim de la base de datos original: (303, 14)

Hay 0 valores perdidos en la variable de salida.

Dim de la base de datos sin las muestras con variable de salida perdido (303, 14)
Se han reseteado a 1 139 valores ya que eran mayores a 0

Procesando imputación de valores perdidos en las características . . .

Imputación finalizada.

No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase


### Aplicando Random Forest con 10 folds

In [2]:
import matplotlib.pyplot as plt

X = DataBase[:,0:13]
Y = DataBase[:,13]

#Normalizamos los datos
X = normalize(X, axis=0, norm='l1')

#Implemetamos la metodología de validación cross validation con 10 folds

#Para calcular el costo computacional
tiempo_i = time.time()

Sensibilidad = np.ones(10)
Especificidad = np.ones(10)
Precision = np.ones(10)
Eficiencia = np.ones(10)
Errores = np.ones(10) 

j = 0
fold_num = 0
kf = KFold(n_splits=10)

m_criterion = "entropy"
m_splitter = "random"
clf = tree.DecisionTreeClassifier(criterion = m_criterion,splitter = m_splitter)

print("Ejecucion con criterion: "+m_criterion+" y splliter: "+m_splitter)
for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  
    fold_num += 1
    #Complete el código aquí creando el modelo de árbol clasificador y entrenandolo unicamente con las muestras de entrenamiento
    #de la base de datos

    
    clf = clf.fit(X_train, y_train)

    #Complete el código aquí prediciendo la clase para las muestras de validación (test) de la base de datos.
    y_pred = clf.predict(X_test)
      
    tn, fp, fn, tp = confusion_matrix(y_test,y_pred ).ravel()
    
    Errores[j] = classification_error(y_pred, y_test)
    Sensibilidad[j] = tp/(tp+fn)
    Especificidad[j] = tn/(tn+fp)
    Precision[j] = tp/(tp+fp)
    Eficiencia[j] = (tp+tn)/(tp+tn+fp+fn)
    j+=1
    
print("\nError de validación: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))
print("\nSensibilidad: "+str(np.mean(Sensibilidad)))
print("\nEspecificidad: "+str(np.mean(Especificidad)))
print("\nPrecision: "+str(np.mean(Precision)))
print("\nEficiencia : "+str(np.mean(Eficiencia)) + "\n ")
print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")






Ejecucion con criterion: entropy y splliter: random

Error de validación: 0.27419354838709675 +/- 0.08087851543083463

Sensibilidad: 0.7072816399286987

Especificidad: 0.7328643157667926

Precision: 0.6984031164913518

Eficiencia : 0.7258064516129031
 


Tiempo total de ejecución: 0.0236690044403 segundos.


<table>
  <tr>
    <th>CRITERION</th>
    <th>SPLITTER</th>
    <th>ERROR DE VALIDACION</th>
    <th>SENSIBILIDAD</th>
    <th>ESPECIFICIDAD</th>
    <th>PRECISION</th>
    <th>EFICIENCIA</th>
    <th>TIEMPO TOTAL DE EJCUCION</th>
    
  </tr>
  <tr>
    <td>gini</td>
    <td>best</td>
    <td>0.2811827956989247 +/- 0.10443429027424364</td>
    <td>0.7021559078176726</td>
    <td>0.7330883912269361</td>
    <td>0.6913932558669401</td>
    <td>0.7188172043010752</td>
    <td>0.0327839851379 segundos.</td>
  </tr>
  <tr>
    <td>gini</td>
    <td>random</td>
    <td>0.2738709677419354 +/- 0.06464154476609289</td>
    <td>0.7063015660809778</td>
    <td>0.7490978294151669</td>
    <td>0.702136393018746</td>
    <td>0.7261290322580646</td>
    <td>0.0201580524445 segundos.</td>
  </tr>
  <tr>
    <td>entropy</td>
    <td>best</td>
    <td>0.303763440860215 +/- 0.09286644283777461</td>
    <td>0.6664915966386554</td>
    <td>0.7191905102121819</td>
    <td>0.6599999999999999</td>
    <td>0.696236559139785</td>
    <td>0.0279319286346 segundos.</td>
  </tr>
  <tr>
    <td>entropy</td>
    <td>random</td>
    <td>0.22139784946236557 +/- 0.047854778006057225</td>
    <td>0.7232505729564553</td>
    <td>0.8241972578505086</td>
    <td>0.785277639027639</td>
    <td>0.7786021505376344</td>
    <td>0.0225191116333 segundos.</td>
  </tr>
</table>

## Selección de Caracteristicas - Wrappers

Para selecionar las caracteristicas usaremos la funcion Wrappers Random Forest. los parámetros son el número de características a seleccionar  (3, 7 y 10) y la estrategia a implementar (SFS, SBS, SFFS, SBFS)

In [3]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import KNeighborsClassifier

#Feature Selection Function
#Recibe 4 parámetros: 1. el modelo (clf para nuestro caso), 2. el número de características final que se quiere alcanzar
#3. Si es forward (True), si es Backward False, 4. Si es es flotante (True), sino False
def select_features(modelo, n_features, fwd, fltg):
    
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=1,
           scoring='accuracy',
           cv=0)
    
    return sfs


#Para calcular el costo computacional
tiempo_i = time.time()

#Implemetamos la metodología de validación cross validation con 10 folds

j = 0
kf = KFold(n_splits=10)

Errores = np.ones(10) 


m_criterion = "entropy"
m_splitter = "random"

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  
   
    #Aquí se entrena y se valida el modelo haciendo selección de características con diferentes estrategias
    
    #Complete el código llamando el método select_features con los parámetros correspondientes para responder el
    #Ejercicio 3.1
    #SFS True False, SBS False False, SFFS True True, SBFS False True
    sf = select_features(clf,7,True,False)

    #Complete el código para entrenar el modelo con las características seleccionadas. Tenga en cuenta
    #la metodología de validación aplicada para que pase las muestras de entrenamiento correctamente.
    sf = sf.fit(X_train,y_train)
    
    
    Errores[j] = 1-sf.k_score_
    j+=1


print("\nError de validación aplicando SFS: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

print("\nEficiencia en validación aplicando SFS: " + str(sf.k_score_*100) + "%" )

print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")

print("Las caracteristicas seleccionadas son: ")

dictChar = {0 : "1. edad", 
            1 : '2. sexo',
            2 : '3. (cp) tipo de dolor en el pecho',
            3 : '4. (trestbps) presión arterial en reposo',
            4 : '5. (chol) colesterol sérico en mg / dl (miligramo por decilitro)',
            5 : '6. (fbs) (azúcar en la sangre en ayunas > 120 mg / dl) (1 = verdadero, 0 = falso)',
            6 : '7. (restecg) resultados electrocardiográficos en reposo:',
            7 : '8. (thalach) frecuencia cardíaca máxima lograda',
            8 : '9. (exang) angina inducida por el ejercicio (1 = sí; 0 = no)',
            9 : '10. (oldpeak) depresión del ST inducida por el ejercicio en relación con el reposo',
            10 :'11. (pendiente) la pendiente del segmento ST de ejercicio máximo',
            11 :'12. (ca) número de vasos principales (0-3) coloreados por fluoroscopia',
            12 :'13. (thal) 3 = normal; 6 = defecto fijo; 7 = defecto reversible'}

for x in range(0,len(sf.k_feature_idx_)):
    print (dictChar[sf.k_feature_idx_[x]])

[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.0s finished
Features: 1/7[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished
Features: 2/7[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.0s finished
Features: 3/7[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
Features: 4/7[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s finished
Features: 5/7[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
Features: 6/7[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s finished
Features: 7/7[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.0s finished
Features: 1/7[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished
Features: 2/7[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.0s finished
Features: 3/7[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
Features: 4/7[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s finished
Features: 5/7[Par


Error de validación aplicando SFS: 0.0 +/- 0.0

Eficiencia en validación aplicando SFS: 100.0%


Tiempo total de ejecución: 1.8153450489 segundos.
Las caracteristicas seleccionadas son: 
1. edad
2. sexo
3. (cp) tipo de dolor en el pecho
4. (trestbps) presión arterial en reposo
5. (chol) colesterol sérico en mg / dl (miligramo por decilitro)
6. (fbs) (azúcar en la sangre en ayunas > 120 mg / dl) (1 = verdadero, 0 = falso)
8. (thalach) frecuencia cardíaca máxima lograda
